# IRE Board members

The goal: Scrape [this list of IRE board members](https://www.ire.org/about-ire/past-ire-board-members/) into a CSV.

This project introduces a few new concepts:
- Scraping data that's not part of a table
- Specifying custom request headers to evade a bot detection rule on our server
- Using string methods and default values when parsing out the data

In [ ]:
# stdlib library we'll use to write the CSV file
import csv

# installed library to handle the HTTP traffic
import requests

# installed library to parse the HTML
from bs4 import BeautifulSoup

In [ ]:
URL = 'https://www.ire.org/about-ire/past-ire-board-members/'

In [ ]:
# set up request headers
# the IRE website rejects incoming requests with the
# `requests` library's default user-agent, so we
# need to pretend to be a browser -- we can do that by
# setting the `User-Agent` value to mimic a value that
# a browser would send, and add this to the headers
# of the request before it's sent
# read more: https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
}

In [ ]:
# send a GET request to fetch the page using the headers we just created
r = requests.get(
    'https://www.ire.org/about-ire/past-ire-board-members/',
    headers=headers
)

# raise an error if the HTTP request returns an error code
# HTTP codes: https://http.cat
r.raise_for_status()

In [ ]:
# use the BeautifulSoup object to parse the response text
# -- r.text -- with the default HTML parser
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#specifying-the-parser-to-use
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
print(soup)

In [ ]:
# search the HTML tree to find the div
# with the `id` attribute of "past-ire-board-members"
target_div = soup.find(
    'div',
    {'id': 'past-ire-board-members'}
)

In [ ]:
print(target_div)

In [ ]:
# within that div, find all the paragraph tags
members = target_div.find_all('p')

In [ ]:
members

In [ ]:
# set up the CSV headers to write to file
csv_headers = [
    'name',
    'terms',
    'was_president',
    'is_deceased'
]

In [ ]:
# next, set up the file to write the CSV data into
# https://docs.python.org/3/library/csv.html#csv.writer

# open the CSV file in write ('w') mode, specifying newline='' to deal with
# potential PC-only line ending problem
with open('ire-board.csv', 'w', newline='') as outfile:

    # set up a csv.writer object tied to the file we just opened
    writer = csv.writer(outfile)

    # write the list of headers
    writer.writerow(csv_headers)

    # loop over the list of paragraphs we targeted above
    for member in members:

        # we don't want the entire Tag object, just the text
        text = member.text

        # set up some default values -- the member was not president
        was_president = False

        # and is not deceased
        is_deceased = False

        # IRE denotes past presidents with a leading asterisk
        # so check to see if the string startswith '*'
        # https://docs.python.org/3/library/stdtypes.html?highlight=startswith#str.startswith
        if text.startswith('*'):

            # if so, switch the value for the `was_president` variable to True
            was_president = True

        # check to see if "(dec)" is anywhere in the text, which
        # indicates this person is deceased
        # https://docs.python.org/3/reference/expressions.html#in
        if '(dec)' in text:
            is_deceased = True

        # next, start parsing out the pieces
        # separate the name from the terms by splitting on "("
        text_split = text.split('(')

        # the name will be the first ([0]) item in the resulting list
        # while we're at it, strip off any leading asterisks
        # https://docs.python.org/3/library/stdtypes.html?highlight=lstrip#str.lstrip
        # and strip() off any leading or trailing whitespace
        # https://docs.python.org/3/library/stdtypes.html?highlight=lstrip#str.strip
        name = text_split[0].lstrip('*').strip()

        # the term(s) of service will be the second item ([1]) in that list
        # and the term text is always terminated with a closing parens
        # so splitting on that closing parens and taking the first ([0])
        # item in the list will give us the term(s)
        terms = text_split[1].split(')')[0]

        # put the collected data into a list
        data = [
            name,
            terms,
            was_president,
            is_deceased
        ]

        # and write this row of data into the CSV file
        writer.writerow(data)